Notebook to send jobs to the Ubelix HPC cluster at the University of Bern

In [ ]:
import sys
import os
import logging
import glob
import pickle
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

logging.basicConfig(filename='example.log', 
                    encoding='utf-8', level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

datapath = Path("../data")
#datapath = Path("/storage/homefs/pd21v747/datanew")

modpath = Path("../scripts")
sys.path.append(os.path.relpath(modpath))

from misc import Timer, pickler, open_table

# when using UBELIX on-demand
os.environ['R_HOME'] = '/storage/homefs/pd21v747/.conda/rna-rep/lib/R/'
import rpy2.robjects as ro
%load_ext rpy2.ipython

In [ ]:
include_unpaired_cancer = False
include_synthetic = False

# 8 cancer types for main results
sites = {"liver": "LIHC",
         "thyroid": "THCA",
         "lung": "LUAD",
         "lung2": "LUSC",
         "kidney": "KIRC",
         "colorectal": "COAD",
         "breast": "BRCA",
         "prostate": "PRAD"}

# Misc unpaired only datasets added in revision
misc_unpaired = {
         "GSETB":"LWPL",
         "yeast":"SNF2"
}

misc_custom_design = {
            "GSEPN":"GIPF",
            "breast_basher": "BASHER", # Basal vs HER2+
            "breast_basluma": "BASLUMA", # Basal vs Luminal A
            "breast_baslumb": "BASLUMB", # Basal vs Luminal B
            "breast_herluma": "HERLUMA", # HER2 vs Luminal A
            "breast_herlumb": "HERLUMB", # HER2 vs Luminal B
            "breast_lumab": "LUMAB" # LumA vs LumB
}

synthetic_sites = {

                  ## SMALL LFC VARIANCE
                  "synthetic10psv": "SBRCA10psv", # 10% DEGs
                  "synthetic25psv": "SBRCA25psv", # 25% DEGs
    
                  ## MEDIUM LFC VARIANCE
                  "synthetic1p":"SBRCA1p", # 1% DEGs
                  "synthetic":"SBRCA1", # 10% DEGS
                  "syntheticl":"SBRCAl", # 25% DEGs

                  ## LARGE LFC VARIANCE
                  "synthetic10plv": "SBRCA10plv" # 10% DEGs

                  # Superfluous
                  # "synthetic5":"SBRCA5", # 5% DEGs
                  # "synthetic2p":"SBRCA2p", # 2% DEGs
                  }

sites = sites | misc_unpaired | misc_custom_design

# Investigate unpaired testing for supplementary figure
if include_unpaired_cancer:
    sites = sites | {k+"_unpaired":v+"_unpaired" for k,v in sites.items()}

if include_synthetic:
    sites = sites | synthetic_sites

datasets = {sites[s]: {} for s in sites}
unpaired_data = list(misc_unpaired.values()) + [s for s in sites.keys() if "unpaired" in s] + list(synthetic_sites.values())

for s in sites:
    f = Path(f"{datapath}/{s}/{sites[s]}/{sites[s]}.csv")
    df = pd.read_csv(f, index_col=0)
    datasets[sites[s]]["genes"] = len(df)
    datasets[sites[s]]["site"] = s
    datasets[sites[s]]["datapath"] = f
    datasets[sites[s]]["metafile"] = f.parent / (f.stem + ".meta.csv")
    datasets[sites[s]]["outpath"] = f.parent
    datasets[sites[s]]["patients"] = len(df.columns)//2
    datasets[sites[s]]["isSynthetic"] = sites[s] in synthetic_sites.values()
    print(f"{s:<10}", datasets[sites[s]]["genes"], datasets[sites[s]]["patients"])
    
# Pretty names
cleanout = {"jk": "ReBoost",
            "pcah": "rPCA",
            "none": "None"}
cleandea = {"edger": "edgeR QLF",
            "edgerlrt": "edgeR LRT",
            "deseq2": "DESeq2",
            "wilcox": "Wilcoxon rank-sum"}

# Differential expression analysis

## Define ground truth

Define ground truth DEGs for a given FDR, logFC cutoff as the intersection of DEGs from all three DEA tests (Wald, LRT, QLF)

In [ ]:
# import sys, importlib
# importlib.reload(sys.modules["DEA"])

from DEA import run_dea_on_full_data
from process import find_ground_truth

DEAs = ["edgerlrt", "edgerqlf", "deseq2", "wilcox"]

# Wilcox was added during revision; truth is defined using original three methods
truth_defining_DEAs = ["edgerlrt", "edgerqlf", "deseq2"]

FDRs = [0.1,0.05,0.01,0.001]
logFCs = [0, 0.5, 1]#, 2] # formal lfc threhsold in edger or deseq2
logFCs_post = [0,0.5,1,1.5,2] # post hoc thresholds

paired = {key:value for key, value in datasets.items() if key not in unpaired_data}
print(f"Running {len(paired)} paired")
#run_dea_on_full_data(paired, DEAs, overwrite = False, lfcs = logFCs, design="paired")

unpaired = {key:value for key, value in datasets.items() if key in unpaired_data}
print(f"Running {len(unpaired)} unpaired")
run_dea_on_full_data(unpaired, DEAs, overwrite = False, lfcs = logFCs, design="unpaired")

custom = {key:value for key, value in datasets.items() if key in misc_custom_design.values()}
print(f"Running {len(custom)} custom designs")
#run_dea_on_full_data(custom, DEAs, overwrite = False, lfcs = logFCs, design="custom")

print("Finding ground truth")
# overwrite True if new dataset added
datasets = find_ground_truth(datasets, truth_defining_DEAs, FDRs, logFCs_post, lfc_tests = logFCs, save=True, overwrite=0) 
#datasets = find_ground_truth(datasets, truth_defining_DEAs, FDRs, [1], lfc_tests = [1])

# Overwrite if new dataset added
#pickler(datasets,"../data/multi/datasets.txt")

In [ ]:
# tab = pd.read_csv("/storage/homefs/pd21v747/RNASeqReplicability/data/breast/BRCA/BRCA.edgerqlf.lfc2.csv", index_col=0)
# tab[tab["FDR"]<0.05]

#datasets["LUAD"]["truth_stats"][0][0.05][0], datasets["LUAD"]["truth_stats"][1][0.05][0]

In [ ]:
# from process import find_ground_truth

# # Calcualte ground truth stats with with Wilcoxon just for one figure
# DEAs_wilcox = ["edgerlrt", "edgerqlf", "deseq2", "wilcox"]
# logFCs_post_wilcox = [0,1]
# datasets_with_wilcox = find_ground_truth(datasets, DEAs_wilcox, FDRs, logFCs_post_wilcox, 
#                                          lfc_tests = [0,1], save=False, overwrite=1) # don't save
# pickler(datasets_with_wilcox, "../data/multi/datasets_wilcox.txt")

## Send batch jobs for selected data set

In [ ]:
selected_data = "HERLUMB"
outpath = datasets[selected_data]["outpath"]
outname = outpath.name
outpath, outname

In [ ]:
from copy import deepcopy
from ubelix import run_multi_batch

script_path = Path("../scripts/send_batch.sh")
DEA_methods = ["edgerqlf","edgerlrt", "deseq2"]#, "wilcox"] # finish edgerqlf jobs before sending other jobs
outlier_methods = ["none"]#, "pcah", "jk"] # only use none for p2 and p3
all_N = [3,4,5,6,7,8,9,10,12,15]
n_cohorts = 100

assert outname in str(outpath)

# lfc 0 threshold, paired
config_params_1 = {
    
    "param_set": "p1", # id for this set of parameters
    "sampler": "paired",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    
    "outlier_kwargs": {
        "none": {},
        "jk": {
            "FDR": 0.01,
            "overwrite": False, # overwrite existing jk tab
            "max_removed_frac": 0.5, # fraction of patients; after 1st iteration, don't jackknife bottom frac patients
            "efficient": True,
            "cols_to_keep": ["FDR"],
            "cleanup": True # remove individual jk tabs and iterations after merger
        },
        "pcah": {"k": 2}
    },
    
    "DEA_kwargs": {
        "edgerqlf": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "design": "paired"},
        "edgerlrt": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "test":"lrt", "design": "paired"},
        "deseq2": {"cols_to_keep": ["logFC","logCPM","FDR"], "design": "paired"},
        "wilcox": {"design": "paired"}
    }
    
}

# lfc = 0 threshold, unpaired
config_params_1u = deepcopy(config_params_1)
config_params_1u["param_set"] = "p10"
config_params_1u["DEA_kwargs"]["edgerqlf"]["design"] = "unpaired"
config_params_1u["DEA_kwargs"]["edgerlrt"]["design"] = "unpaired"
config_params_1u["DEA_kwargs"]["deseq2"]["design"] = "unpaired"
config_params_1u["DEA_kwargs"]["wilcox"]["design"] = "unpaired"

# lfc = 0 threshold, custom
config_params_1c = deepcopy(config_params_1)
config_params_1c["param_set"] = "p1c"
config_params_1c["DEA_kwargs"]["edgerqlf"]["design"] = "custom"
config_params_1c["DEA_kwargs"]["edgerlrt"]["design"] = "custom"
config_params_1c["DEA_kwargs"]["deseq2"]["design"] = "custom"
config_params_1c["DEA_kwargs"]["wilcox"]["design"] = "custom"

# lfc = 1 threshold, paired
config_params_2 = {
    
    "param_set": "p2", # id for this set of parameters
    "sampler": "paired",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    
    "outlier_kwargs": {
        "none": {},
        "jk": {
            "FDR": 0.01,
            "overwrite": False, # overwrite existing jk tab
            "max_removed_frac": 0.5, # fraction of patients; after 1st iteration, don't jackknife bottom frac patients
            "efficient": True,
            "cols_to_keep": ["FDR"],
            "cleanup": True # remove individual jk tabs and iterations after merger
        },
        "pcah": {"k": 2}
    },
    
    "DEA_kwargs": {
        "edgerqlf": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 1, "design": "paired"},
        "edgerlrt": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "test":"lrt", "lfc": 1, "design": "paired"},
        "deseq2": {"cols_to_keep": ["logFC","logCPM","FDR"],"lfc": 1, "design": "paired"}
    }
    
}

# lfc = 1 threshold, custom
config_params_2c = deepcopy(config_params_2)
config_params_2c["param_set"] = "p2c"
config_params_2c["DEA_kwargs"]["edgerqlf"]["design"] = "custom"
config_params_2c["DEA_kwargs"]["edgerlrt"]["design"] = "custom"
config_params_2c["DEA_kwargs"]["deseq2"]["design"] = "custom"
config_params_2c["sampler"] = "unpaired"

# lfc = 1 threshold, unpaired analysis (for data that is unpaired, such as synthetic data)
config_params_3 = {
    
    "param_set": "p3", # id for this set of parameters
    "sampler": "unpaired",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    
    "outlier_kwargs": {
        "none": {},
        "jk": {},
        "pcah": {}
    },
    
    "DEA_kwargs": {
        "edgerqlf": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 1, "design": "unpaired"},
        "edgerlrt": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "test":"lrt", "lfc": 1, "design": "unpaired"},
        "deseq2": {"cols_to_keep": ["logFC","logCPM","FDR"],"lfc": 1, "design": "unpaired"}
    }
    
}

# lfc = 0.5, unpaired
config_params_35 = deepcopy(config_params_3)
config_params_35["param_set"] = "p35"
config_params_35["DEA_kwargs"]["edgerqlf"]["lfc"] = 0.5
config_params_35["DEA_kwargs"]["edgerlrt"]["lfc"] = 0.5
config_params_35["DEA_kwargs"]["deseq2"]["lfc"] = 0.5

# lfc = 2 threshold, paired
config_params_5 = {
    
    "param_set": "p5", # id for this set of parameters
    "sampler": "paired",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    
    "outlier_kwargs": {
        "none": {},
        "jk": {
            "FDR": 0.01,
            "overwrite": False, # overwrite existing jk tab
            "max_removed_frac": 0.5, # fraction of patients; after 1st iteration, don't jackknife bottom frac patients
            "efficient": True,
            "cols_to_keep": ["FDR"],
            "cleanup": True # remove individual jk tabs and iterations after merger
        },
        "pcah": {"k": 2}
    },
    
    "DEA_kwargs": {
        "edgerqlf": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 2, "design": "paired"},
        "edgerlrt": {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "test":"lrt", "lfc": 2, "design": "paired"},
        "deseq2": {"cols_to_keep": ["logFC","logCPM","FDR"],"lfc": 2, "design": "paired"}
    }
    
}

import subprocess as sp
output = sp.getoutput('squeue -u pd21v747')
jobs_running = output.find("send_bat") > 0

#mode = "send jobs" 

# SBATCH with srun does not work from interactive session in newer version of slurm
# https://harvardmed.atlassian.net/wiki/spaces/O2/pages/1586793613/Troubleshooting+Slurm+Jobs#Jobs-fail-with-the-message%3A-Unable-to-satisfy-CPU-bind-request
# workaround: use mode=just testing, copy the command and send job from terminal in submit node
# must set job time/mem manually in shell script

###########################################################################
#### wd must be notebooks folder, must load environment beforehand!!!!#####
###########################################################################

mode = "test main terminal"
mode = "just testing"

import sys, importlib
importlib.reload(sys.modules["ubelix"])
from ubelix import run_multi_batch

do_nothing = False
config_params = config_params_2c

if config_params != config_params_1 and "wilcox" in DEA_methods:
    DEA_methods.remove("wilcox")

if not jobs_running and not do_nothing:
    run_multi_batch(config_params, all_N, n_cohorts, script_path, mode = mode, trysubsample=True)
elif jobs_running:
    print("Jobs running")

In [ ]:
!squeue -u pd21v747

In [ ]:
# tab = open_table("/storage/homefs/pd21v747/RNASeqReplicability/data/liver/LIHC/LIHC_N15/LIHC_N15_0001/tab.none.deseq2.p5.feather")
# tab[tab["FDR"]<0.05]

## Process jobs

In [ ]:
#post hoc
DEAs = ["wilcox", "edgerqlf", "edgerlrt", "deseq2"]
outlier_methods = ["none"]#, "pcah", "jk"]
FDRs = [0.1,0.05,0.01,0.001]
logFCs = [0, 0.5, 1, 1.5, 2]
all_N = [3,4,5,6,7,8,9,10,12,15]
lfc_test = 0
param_set = "p1"

# #formal
DEAs = ["edgerqlf","deseq2", "edgerlrt"]
outlier_methods = ["none"]#, "pcah", "jk"]
FDRs = [0.1,0.05,0.01,0.001]
logFCs = [1]
all_N = [3,4,5,6,7,8,9,10,12,15]
lfc_test = 1
param_set = "p2"

# # #formal
# DEAs = ["edgerqlf","deseq2", "edgerlrt"]
# outlier_methods = ["none"]#, "pcah", "jk"]
# FDRs = [0.1,0.05,0.01,0.001]
# logFCs = [2]
# all_N = [3,4,5,6,7,8,9,10,12,15]
# lfc_test = 2
# param_set = "p5"

#unpaired
DEAs = ["edgerqlf", "edgerlrt", "deseq2"]
outlier_methods = ["none"]
FDRs = [0.05]#[0.1,0.05,0.01,0.001]
logFCs = [1]
all_N = [3,4,5,6,7,8,9,10,12,15]
lfc_test = 1
param_set = "p3"

# #custom
DEAs = ["edgerqlf", "edgerlrt", "deseq2"]
outlier_methods = ["none"]
FDRs = [0.05]#[0.1,0.05,0.01,0.001]
logFCs = [1]
all_N = [3,4,5,6,7,8,9,10,12,15]
lfc_test = 1
param_set = "p2c"

# DEAs = ["edgerqlf", "edgerlrt", "deseq2"]
# outlier_methods = ["none"]
# FDRs = [0.05]#[0.1,0.05,0.01,0.001]
# logFCs = [0]
# all_N = [3,4,5,6,7,8,9,10,12,15]
# lfc_test = 0
# param_set = "p1c"

# #unpaired lfc 0.5
# logFCs = [0.5]
# lfc_test = 0.5
# param_set = "p35"

# #unpaired
# DEAs = ["edgerqlf", "edgerlrt", "deseq2"]
# outlier_methods = ["none"]
# FDRs = [0.05] #[0.1,0.05,0.01,0.001]
# logFCs = [1]
# all_N = [3,7,15]
# lfc_test = 1
# param_set = "p4"

param_sets = ["p1","p1c","p2","p2c","p3","p35","p4","p5"]

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, message='All-NaN slice encountered')


from process import process_pipeline
from misc import profile_func
import pstats

isSynthetic = outname.startswith("SBRCA")

kwargs = {"outpath":outpath, "outname":outname, "all_N": all_N, "DEAs":DEAs, "outlier_methods": outlier_methods, 
          "FDRs":FDRs, "logFCs":logFCs, "lfc_test": lfc_test, "param_set":param_set, "overwrite": 1, "overwrite_merged": 0, "n_cohorts": 100, "isSynthetic": isSynthetic}

do_process = True
do_profile = False

if do_process:
    if do_profile:
        prof = profile_func(process_pipeline, kwargs)
        stats = pstats.Stats(prof).strip_dirs().sort_stats("cumtime")
        stats.print_stats(50)
    else:
        process_pipeline(**kwargs)

In [ ]:
# # Process multiple data sets
# import process, sys, importlib
# importlib.reload(sys.modules["process"])

# from process import process_pipeline

# for data in datasets:
#     print(data)
#     #if not data.startswith("THCA"): continue

#     for key in datasets[data].keys():
        
#         if not isinstance(datasets[data][key], dict):
#             continue
            
#         is_valid_param_set = True
        
#         for N in all_N:
#             if N not in datasets[data][key]:
#                 is_valid_param_set = False
#                 break
                
#         if not is_valid_param_set:
#             continue

#         if key != "p1": continue

#         print(key)

#         if key in ["p1"]:
#             logFCs_k = [0,1] 
#             lfc_test_k = 0
#         elif key in ["p2","p3","p2c","p4"]:
#             logFCs_k = [1]
#             lfc_test_k = 1
#         elif key in ["p35"]:
#             logFCs_k = [0.5]
#             lfc_test_k = 0.5
#         elif key in ["p5"]:
#             logFCs_k = [2]
#             lfc_test_k = 2
#         else:
#             print("Skipping unknown key:", key, data)
#             continue
        
#         outpath = datasets[data]["outpath"]
#         outname = str(outpath).split("/")[-1]
#         kwargs = {"outpath":outpath, "outname":outname, "all_N": all_N, "DEAs":DEAs, "outlier_methods": outlier_methods, 
#                   "FDRs":FDRs, "logFCs":logFCs_k, "lfc_test": lfc_test_k, "param_set":key, "overwrite": 1, "overwrite_merged": 0, "n_cohorts": 100, "isSynthetic": isSynthetic}
#         process_pipeline(**kwargs)

### Merge processed data sets

In [ ]:
def load_results(path, site, name, param_set):
    resultsfile = f"{path}/{site}/{name}/results.{param_set}.txt"
    try:
        with open(resultsfile, "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        pass#print(f"File not found: {resultsfile}")
    
for s in sites:
    #if s not in ["colorectal","prostate","breast","thyroid","liver"]: continue

    for ps in param_sets:
        res = load_results(datapath, s, sites[s], ps)
        if ps not in datasets[sites[s]]:
            datasets[sites[s]][ps] = {}
        if res == None: continue
        for key in res.keys():
            datasets[sites[s]][ps][key] = res[key]

assert "truth_stats" in datasets[list(datasets.keys())[0]] # don't forget to load ground truth
datasetsfile = Path(datapath, "multi/datasets.txt")
pickler(datasets, datasetsfile)

In [ ]:
# Create tidy dataframe

def tidy_df(datasets, param_set, logFCs, all_N, include_sd=False, include_adj=False):
        
    quantity = ["median_rep", "median_deg","median_mcc0", "median_prec0", "median_mcc", "median_prec", "median_rec"]
    
    if include_adj:
        quantity += [q+"_adj" for q in quantity]
    if include_sd:
        quantity += [q+("_sd" if "rep" in q else "_sem") for q in quantity]
    
    quantity += [q+"_syn" for q in quantity]
    quantity += [q+"_method" for q in quantity]
        
    iterables = [datasets,all_N,outlier_methods,DEAs,FDRs,logFCs,quantity]
    multi_cols = pd.MultiIndex.from_product(iterables, names=["Data", "N", "Out", "DEA", "FDR", "logFC", "Val"])
    combined = pd.DataFrame(columns=multi_cols)

    ids = [] # id connecting samples from same dataset and cohort size, used for paired-sample testing pre vs. post outlier removal
    for i, d in enumerate(datasets):
        #if d not in ["COAD","PRAD","BRCA","THCA","LIHC"]: continue
        isSynthetic = datasets[d]["isSynthetic"]
        for j, N in enumerate(all_N):
            k = i*len(all_N)+j 
            for out in outlier_methods:
                for dea in DEAs:
                    for fdr in FDRs:
                        for logFC in logFCs:
                            for quant in quantity:
                                if "_syn" in quant:# and not isSynthetic: 
                                    continue
                                col = (d,N,out,dea,fdr,logFC,quant)
                                try:
                                    combined.loc[0,col] = datasets[d][param_set][N][out][dea][quant][fdr][logFC]
                                except KeyError:
                                    if "syn_hom" in d and out != "None": pass
                                    else: 
                                        print(d,N,param_set,out,dea,quant,fdr,logFC)
                                        combined.loc[0,col] = datasets[d][param_set][N][out][dea][quant][fdr][logFC]
                                        raise Exception("KeyError")
                                if quant == "median_rep": ids.append(k)

    combined_td = combined.unstack().unstack(level="Val").reset_index(level=["Data","N","Out","DEA","FDR","logFC"], drop=False)
    combined_td["id"] = ids
    combined_td.reset_index(drop=True, inplace=True)

    # For outlier_method == "none", no adjustment is necessary, hence copy unadjusted values
    if include_adj:
        none_ix = combined_td[combined_td["Out"] == "none"].index # avoid setting with copy warning
        combined_td.loc[none_ix,"median_deg_adj"] = combined_td.loc[none_ix,"median_deg"]
        combined_td.loc[none_ix,"median_rep_adj"] = combined_td.loc[none_ix,"median_rep"]
        combined_td.loc[none_ix,"median_mcc_adj"] = combined_td.loc[none_ix,"median_mcc"]
        combined_td.loc[none_ix,"median_mcc0_adj"] = combined_td.loc[none_ix,"median_mcc0"]
        combined_td.loc[none_ix,"median_prec_adj"] = combined_td.loc[none_ix,"median_prec"]
        combined_td.loc[none_ix,"median_prec0_adj"] = combined_td.loc[none_ix,"median_prec0"]
        combined_td.loc[none_ix,"median_rec_adj"] = combined_td.loc[none_ix,"median_rec"]


    for clean in cleanout:
        combined_td.loc[(combined_td[combined_td["Out"] == clean]).index, "Out"] = cleanout[clean]
    for clean in cleandea:
        combined_td.loc[(combined_td[combined_td["DEA"] == clean]).index, "DEA"] = cleandea[clean]
    return combined_td.infer_objects()

In [ ]:
# outlier_methods = ["none","jk","pcah"]
# combined_td = tidy_df("p1",[0,1])
# outlier_methods = ["none"]
# combined_td2 = tidy_df("p2",[1])
# combined_td.head()

In [ ]:
#d = {k: datasets[k] for k in ("PRAD_unpaired","BRCA_unpaired","THCA_unpaired","LIHC_unpaired","LUAD_unpaired","COAD_unpaired","KIRC_unpaired")}
#d = {k: datasets[k] for k in ("GIPF","LWPL","PRAD","BRCA","THCA","LIHC","LUAD","COAD","KIRC")}
#d = datasets

#d = {k: datasets[k] for k in ("PRAD","BRCA","THCA","LIHC","LUAD","LUSC","COAD","KIRC")} # p1, p2
#d = {k: datasets[k] for k in ["SBRCA1","SBRCAs","SBRCAl","SBRCA5","SBRCA2p","SBRCA1p"]}
d = {k: datasets[k] for k in ["SBRCA1","SBRCAl","SBRCA1p","SBRCA10plv", "LWPL","SNF2"]} # p3 # "LUMAB"
d = {k: datasets[k] for k in ["HERLUMA","BASHER","BASLUMA","BASLUMB","GIPF","LUMAB","HERLUMB"]} # p2c
#d = {k: datasets[k] for k in ["LUMAB"]} # p1c
#d = {k: datasets[k] for k in ["SBRCA10psv","SBRCA25psv"]} # p35

##### Post hoc fold change threshold

# param_set = "p1"
# combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[0,1], include_sd=False)

# param_set = "p1c"
# combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[0], include_sd=False)

##### Formal fold change threshold

#param_set = "p2"
#combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[1])

# param_set = "p5"
# #combined_td = tidy_df(d, param_set, all_N=all_N,, logFCs=[2])

#### Formal unpaired

# param_set = "p3"
# combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[1], include_sd=False)

#param_set = "p35"
#combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[0.5])

#### Formal custom

param_set = "p2c"
combined_td = tidy_df(d, param_set, all_N=all_N, logFCs=[1], include_sd=False)


##### Save

combined_td.to_csv(Path(datapath, f"multi/combined_td.{param_set}.csv"))
combined_td

In [ ]:
d = {k: datasets[k] for k in ["GIPF","LWPL"]}
param_set = "p3"
combined_td_gse = tidy_df(d, param_set, logFCs=[1], all_N_alt=[3,7,15],include_sd=False)
combined_td_gse.to_csv(Path(datapath, f"multi/combined_td.gse.{param_set}.csv"))

In [ ]:
combined_td2 = pd.read_csv("../data/multi/combined_td.p2.csv", index_col=0) # Formal fold change thresholds, paired
combined_td3 = pd.read_csv("../data/multi/combined_td.p3.csv", index_col=0) # Formal fold change thresholds, unpaired

## Inspect results

In [ ]:
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/kidney/KIRC/KIRC_N15/KIRC_N15_0005/tab.none.edgerlrt.p3.feather"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSETB/LWPL/LWPL_N3/LWPL_N3_0005/tab.none.edgerlrt.p3.feather"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/prostate/PRAD/PRAD_N3/PRAD_N3_0007/tab.none.edgerlrt.p3.feather"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEPN/GIPF/GIPF_N3/LWPL_N3_0005/tab.none.edgerlrt.p3.feather"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/lung2/LUSC/LUSC_N3/LUSC_N3_0007/tab.none.edgerlrt.p3.feather"

tab = open_table(f)
sig = tab[tab["FDR"]<0.05]
print(len(sig))
order = tab.sort_values(by="logFC").index
tab.loc[order]

In [ ]:
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEBP/BPLT/BPLT.edgerlrt.lfc0.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEBP/BPLT/BPLT.edgerlrt.lfc0.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/prostate/PRAD/PRAD.edgerlrt.lfc0.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/lung2/LUSC/LUSC.edgerlrt.lfc0.csv"
truth = open_table(f)
sig_truth = truth[truth["FDR"]<0.05]
print(len(sig_truth))
truth.loc[order]

In [ ]:
plt.hist(truth["logFC"],bins=30, range=(-3,3),alpha=0.5,label="Truth")
plt.hist(tab["logFC"],bins=30, range=(-3,3),alpha=0.5,label="Cohort")
plt.legend()

In [ ]:
sns.jointplot(x=tab.loc[order,"logFC"], y=truth.loc[order,"logFC"], kind="reg", line_kws=dict(color="r"))
plt.plot((-3,3),(-3,3),ls="--",c="black")

In [ ]:
if "combined_td3" not in globals():
    combined_td3 = pd.read_csv("../data/multi/combined_td.p3.csv", index_col=0) # Unpaired formal lfc 1
    #combined_td3["DEA"] = combined_td3["DEA"].str.replace("edgerqlf","edgeR QLF")

In [ ]:
combined_td3["DEA"].value_counts()

In [ ]:
plot_set = tidy_df({"THCA": datasets["THCA"]} | {"BRCA": datasets["BRCA"]} | {"LUSC":datasets["LUSC"]} 
                   | {"LIHC":datasets["LIHC"]} | {"PRAD":datasets["PRAD"]}
                    | {"LUAD":datasets["LUAD"]} | {"COAD": datasets["COAD"]} 
                   | {"KIRC":datasets["KIRC"]}, "p1", logFCs=[0,1])

#plot_set.to_csv("../data/multi/combined_td_wilcox.p1.csv")
#plot_set = tidy_df(datasets, "p1", [0,1])
#plot_set = combined_td3

In [ ]:
tab = pd.read_csv("/storage/homefs/pd21v747/RNASeqReplicability/data/breast_lumab/LUMAB/LUMAB.edgerqlf.lfc1.csv", index_col=0)
sigt = tab[tab["FDR"]<0.05]
len(sigt)

In [ ]:
N = 3
c = 4
f = f"/storage/homefs/pd21v747/RNASeqReplicability/data/breast_lumab/LUMAB/LUMAB_N{N}/LUMAB_N{N}_{c:04}/tab.none.edgerqlf.p3.feather"

df = open_table(f)
sig = df[df["FDR"]<0.05]
print(len(sig))

sig.index.intersection(sigt.index)

In [ ]:
from misc import get_grid_size

plot_set = combined_td

x = "N"
y = "median_prec_method"
y = "median_rec"

a  = plot_set#[plot_set["logFC"]==1]
#a = a[a["Data"].str.startswith("LUSC")]
a = a[a["FDR"]==0.05]
a = a[a["Out"]=="None"]
a = a[a["Data"]=="SNF2"]

hues = ["Data","logFC","DEA","Out","N","FDR"]
grid = (2,3)#get_grid_size(len(hues), k=0, fill=True)
fig, ax = plt.subplots(grid[0],grid[1],figsize=(6*grid[0],3*grid[1]), sharex=True,sharey=True)
ax=ax.flatten()
cube = sns.cubehelix_palette(as_cmap=False, n_colors=5)

for i, hue in enumerate(hues):
    sns.scatterplot(data=a, x=x, y=y, hue=hue, ax=ax[i])
    #ax[i].plot((0.8,1),(0.8,1),ls="--")
#for axx in ax: axx.invert_yaxis()
fig.tight_layout()

In [ ]:
sns.set(font_scale=1.7)

fig, ax = plt.subplots(2,3,figsize = (24,16))
ax = ax.flatten()

plot_set = combined_td

a = plot_set#[plot_set["logFC"]==0]
a = a[a["FDR"]==0.05]
a = a[a["Out"]=="None"]

sns.boxplot(data=a, x="N", y="median_mcc",hue="DEA", ax=ax[0])
sns.boxplot(data=a, x="N", y="median_prec",hue="DEA", ax=ax[1])
sns.boxplot(data=a, x="N", y="median_rec",hue="DEA", ax=ax[2])


plot_set2 = combined_td2
a  = plot_set2[plot_set2["logFC"]==1]
a = a[a["FDR"]==0.05]
a = a[a["Out"].isin(["None",np.nan])]

sns.boxplot(data=a, x="N", y="median_mcc0",hue="DEA", ax=ax[3])
sns.boxplot(data=a, x="N", y="median_prec",hue="DEA", ax=ax[4])
sns.boxplot(data=a, x="N", y="median_rec",hue="DEA", ax=ax[5])

for i in range(len(ax)):
    if i != 3:
         ax[i].legend().remove()

sns.set(font_scale=1)

In [ ]:
sns.set(font_scale=1.7)

fig, ax = plt.subplots(2,3,figsize = (24,16))
ax = ax.flatten()

plot_set = combined_td

a = plot_set[plot_set["logFC"]==1]
a = a[a["FDR"]==0.05]
a = a[a["Out"]=="None"]

sns.boxplot(data=a, x="N", y="median_mcc",hue="DEA", ax=ax[0])
sns.boxplot(data=a, x="N", y="median_prec",hue="DEA", ax=ax[1])
sns.boxplot(data=a, x="N", y="median_rec",hue="DEA", ax=ax[2])


plot_set2 = combined_td
a  = plot_set2[plot_set2["logFC"]==1]
a = a[a["FDR"]==0.05]
a = a[a["Out"].isin(["None",np.nan])]

sns.boxplot(data=a, x="N", y="median_mcc_syn",hue="DEA", ax=ax[3])
sns.boxplot(data=a, x="N", y="median_prec_syn",hue="DEA", ax=ax[4])
sns.boxplot(data=a, x="N", y="median_rec_syn",hue="DEA", ax=ax[5])

for i in range(len(ax)):
    if i != 3:
         ax[i].legend().remove()

sns.set(font_scale=1)

In [ ]:
combined_td[[c for c in combined_td.columns if combined_td[c].iloc[0] not in ["None",None]]]

In [ ]:
plot_set = pd.read_csv("../data/multi/combined_td.p3.csv", index_col=0) # UNpaired, synthetic data


In [ ]:
sns.set(font_scale=2)
sns.set_style("ticks")

plot_set = combined_td
plot_set["Out"].fillna("None", inplace=True)
plot_set["Data"].replace({"SBRCAl": "Synthetic, 25% DEGs",
                          "SBRCA1": "Synthetic, 10% DEGs", 
                          "SBRCAs": "Synthetic, 1% DEGs"}, inplace=True)

fig, ax = plt.subplots(4,3,figsize = (24,32), sharey="row", sharex=True)
sns.despine()
ax = ax.flatten()


a = plot_set[plot_set["logFC"]==1]
a = a[a["FDR"]==0.05]
#a = a[a["DEA"]=="edgerqlf"]
#a = a[a["DEA"]=="edgeR LRT"]
a = a[a["Out"]=="None"]

m = np.array(range(12)).reshape((4,3)).T.flatten() # transpose plot grid
deas = ["DESeq2","edgerqlf","edgeR LRT"]
for i, dea in enumerate(deas):
    b = a[a["DEA"]==dea]
    sns.lineplot(data=b, x="N", y="median_mcc",hue="Data", marker="o", ms=20, ax=ax[m[0+i*4]])#ax=ax[0+i*3])
    sns.lineplot(data=b, x="N", y="median_prec",hue="Data", marker="o", ms=20, ax=ax[m[1+i*4]])#ax=ax[1+i*3])
    sns.lineplot(data=b, x="N", y="median_rec",hue="Data", marker="o", ms=20, ax=ax[m[2+i*4]])#ax=ax[2+i*3])
    sns.lineplot(data=b, x="N", y="median_rep",hue="Data", marker="o", ms=20, ax=ax[m[3+i*4]])#ax=ax[3+i*3])
    ax[i].set(title=prdea[dea] if "prdea" in globals() and dea in prdea else dea)

ax[0].set_ylabel("Median MCC")
ax[3].set_ylabel("Median Precision")
ax[6].set_ylabel("Median Recall")
ax[9].set_ylabel("Median Replicability")

for i, a in enumerate(ax):
    a.legend(title=None)
    if i % 3 == 0:
        #a.set_xticks(ticks=all_N, labels=all_N)
        a.set_xticks(ticks=range(3,16), labels=range(3,16))
        
fig.tight_layout()
figpath = f"../figures/sfig28_simulated_data_lfc1.pdf"
fig.savefig(figpath)

In [ ]:
sns.set(font_scale=1.7)

fig, ax = plt.subplots(2,3,figsize = (24,16))
ax = ax.flatten()

plot_set = combined_td

a = plot_set#[plot_set["logFC"]==0]
a = a[a["FDR"]==0.05]
a = a[a["Out"]=="None"]

sns.boxplot(data=a, x="N", y="median_mcc",hue="DEA", ax=ax[0])
sns.boxplot(data=a, x="N", y="median_prec",hue="DEA", ax=ax[1])
sns.boxplot(data=a, x="N", y="median_rec",hue="DEA", ax=ax[2])


plot_set2 = combined_td2
a  = plot_set2[plot_set2["logFC"]==1]
a = a[a["FDR"]==0.05]
a = a[a["Out"].isin(["None",np.nan])]

sns.boxplot(data=a, x="N", y="median_mcc",hue="DEA", ax=ax[3])
sns.boxplot(data=a, x="N", y="median_prec",hue="DEA", ax=ax[4])
sns.boxplot(data=a, x="N", y="median_rec",hue="DEA", ax=ax[5])

for i in range(len(ax)):
    if i != 3:
         ax[i].legend().remove()

sns.set(font_scale=1)

In [ ]:
c=combined_td
c=c[(c["FDR"]==0.05)]
c

# Function to calculate weighted mean and CI for each sample size
def calculate_weighted_mean(group, col):
    vals = group[col].values
    sems = group[col+("_sd" if "rep" in col else "_sem")].values
    
    # Calculate variances from SEMs
    variances = sems ** 2
    
    # Calculate weights as the inverse of variances
    weights = 1 / variances
    
    # Calculate weighted mean recall
    weighted_mean = np.sum(weights * vals) / np.sum(weights)
    
    # Calculate the variance of the weighted mean
    summary_variance = 1 / np.sum(weights)
    
    # Calculate the summary CI (95% CI, using 1.96)
    summary_CI = 1.96 * np.sqrt(summary_variance)
    
    return pd.Series({
        f"{col}": weighted_mean,
        "CI": summary_CI
    })

val = "median_deg"
# Group by "Sample Size" and apply the function
result = c.groupby(["DEA","N"]).apply(lambda x: calculate_weighted_mean(x, val)).reset_index()
#result

In [ ]:
for i, dea in enumerate(set(result["DEA"])):
    r = result
    r = r[r["DEA"]==dea]

    lower_bound = r[val] - r["CI"]
    upper_bound = r[val] + r["CI"]
    
    # Clip the bounds to [0, 1]
    lower_bound_clipped = np.clip(lower_bound, 0, (1 if "deg" not in val else np.inf))
    upper_bound_clipped = np.clip(upper_bound, 0, (1 if "deg" not in val else np.inf))
    
    # Calculate the clipped yerr for plt.errorbar
    yerr_clipped = [r[val] - lower_bound_clipped, upper_bound_clipped - r[val]]


    offset = -0.3+i*0.3
    plt.errorbar(x=r["N"]+offset, y=r[val], 
                 #yerr=r["CI"],
                 #yerr=[r["CI"],r["CI"] - np.max(0,r[val])], 
                 yerr = yerr_clipped,
                 markersize=10,fmt="o")

In [ ]:
r = result
r = r[r["DEA"]=="DESeq2"]

In [ ]:
plt.figure(figsize=(8, 6))
def f(x):
    print(x)
    return(0,0)
sns.pointplot(x="N", y="Weighted Mean median_prec", data=r, errorbar=lambda x: f(x))
plt.ylabel("Weighted Mean Recall")
plt.title("Weighted Mean Recall with Summary CI by Sample Size")
plt.show()

In [ ]:
datasets["PRAD"]["p1"][15]["none"]["wilcox"]["median_deg"][0.05][1]

# Enrichment analysis

## Ground truth definition

Run GSEA on ground truth logFC estimates

In [ ]:
from enrichment import prepare_gsea, run_gseapy_libraries, convert_ids_biomart, find_conv_table_all, clean_tab

libraries = [
  "GO_Biological_Process_2021",
  "KEGG_2021_Human"
  # "MSigDB_Oncogenic_Signatures",
  # "Cancer_Cell_Line_Encyclopedia",
  # "GO_Molecular_Function_2021",
  # "GeneSigDB",
  # "GO_Cellular_Component_2021",
  # "MSigDB_Hallmark_2020",
  # "MSigDB_Computational",
  # "WikiPathway_2021_Human"
]

#cancers = ["PRAD","COAD","BRCA","KIRC","LIHC","LUAD","THCA","LUSC"]

datasetsfile = Path(datapath, "multi/datasets.txt")
with open(datasetsfile, "rb") as f:
    datasets = pickle.load(f)

# Create table for converting ENSG to Gene Symbols and Entrez IDs
conv_file = Path(datapath, "multi/conv_table.csv")
find_conv_table_all(datasets, conv_file, overwrite=False)
conv_table = pd.read_csv(conv_file, index_col=0)
conv_table.head()

### GSEApy

In [ ]:
from enrichment import prepare_gsea, run_gseapy_libraries, convert_ids_biomart, find_conv_table_all, clean_tab
from process import signal_to_noise
from DEA import normalize_counts
            
overwrite_all_gsea = False

# lfc estimates are similar between edegR and deseq2, so we focus on one tool
method = "deseq2"

for k, data in enumerate(datasets):
    
    if datasets[data]["isSynthetic"]: continue

    if data == "SNF2": 
        continue # skipping non-human data (yeast)

    outpath = datasets[data]["outpath"]
    gseapath = f"{outpath}/gsea"   

    #os.system(f"mkdir {gseapath}")

    ### Load ground truth lfc estimates
    #tab = pd.read_csv(f"{outpath}/truth_lfc.csv", index_col=0) # truth based on methods consensus
    tab = pd.read_csv(f"{outpath}/{data}.{method}.lfc0.csv", index_col=0) # method-specific truth

    tab.dropna(inplace=True)
    
    ### Calcualte S2N
    #counts = pd.read_csv(datasets[data]["datapath"], index_col=0)
    #counts = normalize_counts(counts)
    #tab.loc[counts.index.intersection(tab.index), "|S2N|"] = signal_to_noise(counts)

    if tab.index[0].startswith("ENSG"):
        tab_cleaned = tab.loc[tab.index.intersection(conv_table.index)]
        tab_cleaned["Symbol"] = conv_table.loc[tab_cleaned.index,"Symbol"]
    else: # assume symbol
        tab_cleaned = tab.loc[tab.index.intersection(conv_table.set_index("Symbol").index)]
        tab_cleaned["Symbol"] = conv_table.set_index("Symbol").loc[tab_cleaned.index].index
        
    logging.info(f"{data}\nOriginal tab: {len(tab)} genes\nCleaned tab: {len(tab_cleaned)} genes\n")

    with Timer(name="context manager"):        
        run_gseapy_libraries(tab_cleaned, gseapath, libraries, overwrite_all_gsea, permutation_num=1000, save_full_results=True, ranking="logFC", min_size=15, max_size=500, file_id=method)
    
        #run_gseapy_libraries(tab_cleaned, gseapath, libraries, overwrite_all_gsea, permutation_num=1000, save_full_results=True, ranking="|S2N|", min_size=15, max_size=500)

In [ ]:
# Inspect results

gsea_out = f"{gseapath}/gseapy.logFC.{libraries[1]}.{method}.txt"
with open(gsea_out, "rb") as fp:
    gsea_results = pickle.load(fp)
    
print(gsea_out)

In [ ]:
import gseapy

terms = gsea_results.res2d.sort_values(by="FDR q-val")
display(gsea_results.res2d.head(10))
top_term = terms["Term"][0]
print(f"Total gene sets tested: {len(terms)}")
print(f"Significant gene sets at 5% FDR: {len(terms[terms['FDR q-val']<0.05])}\n")
gseapy.gseaplot(rank_metric=gsea_results.ranking, term=top_term, **gsea_results.results[top_term])

In [ ]:
# Make results dictionary

FDR = 0.05
rankings = ["logFC"]#, "|S2N|"]
gsea_dict = {data: {"truth": {"gseapy": {rnk: {lib: {} for lib in libraries} for rnk in rankings}}} for data in datasets if not datasets[data]["isSynthetic"] and data!="SNF2"}

for data in datasets:
    
    outpath = datasets[data]["outpath"]
    gseapath = f"{outpath}/gsea"
    
    for rnk in rankings:
    
        for library in libraries:

            reportfile = f"{gseapath}/gseapy.{rnk}.{library}.{method}.feather"
            try:
                terms = open_table(reportfile).sort_values(by="FDR")
            except FileNotFoundError:
                continue
            gsea_dict[data]["truth"]["gseapy"][rnk][library] = terms
            print(data, library, rnk, len(terms), len(terms[terms['FDR']<FDR]))

### clusterProfiler ORA  (derpecated)

In [ ]:
from enrichment import run_clusterORA, convert_ensg

overwrite_clusterORA = False
FDRs = [0.05]
logFCs = [0,1] # formal lfc threshold

ORA_kwargs = {"FDRs": [0.05], "logFCs": [0,1], "minGSSize": 15, "maxGSSize": 500,
               "use_internal_data": True, "internal_data_path": "../data/clusterORA/KEGG_DATA.RData"}

for k, data in enumerate(datasets):
    print(data)
    
    outpath = datasets[data]["outpath"]
    gseapath = f"{outpath}/gsea"
    #os.system(f"mkdir {gseapath}")
    universe = pd.read_csv(f"{outpath}/{data}.csv", usecols=["Unnamed: 0"], index_col=0)
    universe = list(convert_ensg(universe,conv_table,target="Entrez").index)

    for fdr in FDRs:
        for logFC in logFCs:
            degs = pd.read_csv(f"{outpath}/truth.fdr{fdr}.post_lfc{logFC}.lfc{logFC}.csv", usecols=["Unnamed: 0"], index_col=0)
            degs = list(convert_ensg(degs,conv_table,target="Entrez").index)
    
            s = "_lfc" if logFC > 0 else ""
            prefix = f"{gseapath}/clusterORA{s}.fdr{fdr}.post_lfc{logFC}.lfc{logFC}"
            run_clusterORA(degs, universe, file_id="", go_ont="BP", prefix=prefix, overwrite=overwrite_clusterORA, **ORA_kwargs)

In [ ]:
for data in datasets:
    
    outpath = datasets[data]["outpath"]
    gseapath = f"{outpath}/gsea"
        
    for fdr in FDRs:
        for logFC in logFCs:
            
            s = "_lfc" if logFC > 0 else ""
            method = f"clusterORA{s}.fdr{fdr}.post_lfc{logFC}.lfc{logFC}"
            gsea_dict[data]["truth"][method] = {lib: None for lib in libraries}
            
            for library in libraries:
                #if "KEGG" not in library: continue
                    
                reportfile = f"{gseapath}/{method}.{library}.feather"         
                terms = open_table(reportfile).sort_values(by="FDR")
                
                ## for KEGG: switch term ID and description for index
                if terms.index[0].startswith("hsa") and "Term" in terms:
                    terms["Term ID"] = terms.index
                    terms = terms.set_index("Term")
                
                gsea_dict[data]["truth"][method][library] = terms
                print(data, method, library, len(terms), len(terms[terms['FDR']<FDR]))

In [ ]:
# For KEGG ORA,set term name as index instead of term ID

for site in sites:
    for fdr in FDRs:
        for lfc in logFCs:
            s = "_lfc" if lfc > 0 else ""
            method = f"clusterORA{s}.fdr{fdr}.post_lfc{lfc}.lfc{lfc}"
            f = f"../data/{site}/{sites[site]}/gsea/{method}.KEGG_2021_Human.feather"
            t=open_table(f)
            if "Term ID" in t or t.index.name == "Term": continue
            t["Term ID"] = t.index
            t.set_index("Term",inplace=True)
            t = t.reset_index()
            t.to_feather(f)
            display(t)

### Common terms (derpecated)

We will define an additional, common ground truth, restricted to terms to terms that are shared between GSEApy, clusterProfiler ORA and all data sets.

In [ ]:
def get_common_terms():
    common_kegg, common_gobp = set(), set()
    for data in gsea_dict:
        for lfc in [0,1]:
            s = "_lfc" if lfc == 1 else ""
            # KEGG
            gseapy_terms = gsea_dict[data]["truth"]["gseapy"]["logFC"]["KEGG_2021_Human"].index
            clusterORA_terms = set(gsea_dict[data]["truth"][f"clusterORA{s}.fdr0.05.post_lfc{lfc}.lfc{lfc}"]["KEGG_2021_Human"].index)

            if len(common_kegg)<1: common_kegg = gseapy_terms.intersection(clusterORA_terms)
            else: common_kegg = common_kegg.intersection(gseapy_terms).intersection(clusterORA_terms)

            # GO BP
            gseapy_terms = gsea_dict[data]["truth"]["gseapy"]["logFC"]["GO_Biological_Process_2021"].index
            clusterORA_terms = gsea_dict[data]["truth"][f"clusterORA{s}.fdr0.05.post_lfc{lfc}.lfc{lfc}"]["GO_Biological_Process_2021"].index    

            if len(common_gobp)<1: common_gobp = gseapy_terms.intersection(clusterORA_terms)
            else: common_gobp = common_gobp.intersection(gseapy_terms).intersection(clusterORA_terms)

    return common_kegg, common_gobp
    
file_gobp = Path("../data/multi/common_gobp.txt")
file_kegg = Path("../data/multi/common_kegg.txt")
if (not file_gobp.is_file() or not file_kegg.is_file()):
    common_kegg, common_gobp = get_common_terms()
    pickler(common_gobp, file_gobp)
    pickler(common_kegg, file_kegg)
else:
    with open(file_gobp, "rb") as f:
        common_gobp = pickle.load(f)
    with open(file_kegg, "rb") as f:
        common_kegg = pickle.load(f)
        
print("KEGG Terms:", len(common_kegg))
print("GO BP Terms:", len(common_gobp))

As we have restricted the number of terms (hypotheses), we need to recalculate the adjusted pvalues for appropriate FDR control.

In [ ]:
from statsmodels.stats.multitest import multipletests

redo_common_fdr = True
FDRs = [0.05]
logFCs = [0,1]
rankings = ["logFC", "|S2N|"]

if redo_common_fdr:

    for data in datasets:

        outpath = datasets[data]["outpath"]
        gseapath = f"{outpath}/gsea"
        
        for library in libraries:
            
            common = common_gobp if library == "GO_Biological_Process_2021" else common_kegg
    
            ### GSEA
            for rnk in rankings:
                suffix = "" if rnk == "logFC" else "_s2n"
                reportfile = f"{gseapath}/gseapy{suffix}.{library}.feather"
                terms = open_table(reportfile).sort_values(by="FDR")
                terms["FDR.common"] = np.full(len(terms),np.nan)
                terms.loc[common, "FDR.common"] = multipletests(terms.loc[common, "NOM p-val"], method="fdr_bh")[1]
                terms.reset_index().to_feather(reportfile)


            ### ORA
            for fdr in FDRs:
                for logFC in logFCs:
                    s = "_lfc" if logFC > 0 else ""
                    method = f"clusterORA{s}.fdr{fdr}.post_lfc{logFC}.lfc{logFC}"

                    reportfile = f"{gseapath}/{method}.{library}.feather"         
                    terms = open_table(reportfile).sort_values(by="FDR")
                    #assert terms.index.duplicated().sum() == 0
                    terms["FDR.common"] = np.full(len(terms),np.nan)
                    
                    if terms.index[0].startswith("hsa"):
                        terms["hsa"] = terms.index
                        terms.set_index("Term", inplace=True)
                    
                    ix = common #if library == "GO_Biological_Process_2021" else terms[terms["Term"].isin(common)].index
                    print(data,library,fdr,logFC,method)
                    terms.loc[ix, "FDR.common"] = multipletests(terms.loc[ix, "pvalue"], method="fdr_bh")[1]
                    terms.reset_index().to_feather(reportfile)

In [ ]:
gsea_methods = ['gseapy', 'gseapy_s2n',
               'clusterORA.fdr0.05.post_lfc0.lfc0',
               'clusterORA_lfc.fdr0.05.post_lfc1.lfc1']


from process import get_n_gsea_truth
FDRs=[0.05]
gsea_param_set = "p1"
gsea_truth = {data: {out: {dea: {gsea: {lib: {"truth"+mode: {fdr: None for fdr in FDRs} for mode in ["","_common"]} for lib in libraries} for gsea in gsea_methods} for dea in DEA_methods} for out in outlier_methods} for data in datasets if "syn" not in data}

for data in datasets:
    print(data)
    outpath_d = datasets[data]["outpath"]
    outname_d = str(outpath_d).split("/")[-1]
    gsea_truth[data] = get_n_gsea_truth(gsea_truth[data], outpath_d, outname_d, all_N, DEA_methods, outlier_methods, gsea_methods, libraries, FDRs, gsea_param_set, overwrite=False)


In [ ]:
gsea_datasets = {sites[s]: {} for s in sites if sites[s] in cancers}

DEAs = DEA_methods
FDRs = [0.05]
all_N=[3]#,5,7,9,15]
    
quantity = ["truth"]
quantity += [q+"_common" for q in quantity]

iterables = [gsea_datasets,outlier_methods,DEAs,gsea_methods,libraries,FDRs,quantity]
multi_cols = pd.MultiIndex.from_product(iterables, names=["Data", "Out", "DEA", "Enrichment", "Library", "FDR", "Val"])
gsea_truth_df = pd.DataFrame(columns=multi_cols)

ids = [] # id connecting samples from same dataset and cohort size, used for paired-sample testing pre vs. post outlier removal
for d in gsea_datasets:
    for out in outlier_methods:
        for dea in DEAs:
            for gsea in gsea_methods:
                #if dea != "edger" and gsea_methods == "gsea_s2n": continue
                for fdr in FDRs:
                    for lib in libraries:
                        for quant in quantity:
                            #print(d,N,out,dea,fdr)
                            col = (d,out,dea,gsea,lib,fdr,quant)
                            if dea != "edgerqlf" and "s2n" in gsea: dea_source = "edgerqlf"
                            else: dea_source = dea
                            gsea_truth_df.loc[0,col] = gsea_truth[d][out][dea_source][gsea][lib][quant][fdr]
                            
gsea_truth_df = gsea_truth_df.unstack().unstack(level="Val").reset_index(level=["Data","Out","DEA","Enrichment","Library","FDR"], drop=False)
gsea_truth_df.reset_index(drop=True, inplace=True)

# For outlier_method == "none", no adjustment is necessary, hence copy unadjusted values
#none_ix = combined_gsea_td[combined_gsea_td["Out"] == "none"].index # avoid setting with copy warning
#combined_gsea_td.loc[none_ix,"median_terms_adj"] = combined_gsea_td.loc[none_ix,"median_terms"]
#combined_gsea_td.loc[none_ix,"median_rep_adj"] = combined_gsea_td.loc[none_ix,"median_rep"]

for clean in cleanout:
    gsea_truth_df.loc[(gsea_truth_df[gsea_truth_df["Out"] == clean]).index, "Out"] = cleanout[clean]
for clean in cleandea:
    gsea_truth_df.loc[(gsea_truth_df[gsea_truth_df["DEA"] == clean]).index, "DEA"] = cleandea[clean]
gsea_truth_df.to_csv(f"../data/multi/gsea_truth_df.csv")
gsea_truth_df.head()

## Send batch jobs

In [ ]:
selected_data = "LUAD"
outpath = datasets[selected_data]["outpath"]
outname = outpath.name
outpath, outname

In [ ]:
import sys, importlib
importlib.reload(sys.modules["enrichment"])
importlib.reload(sys.modules["ubelix"])

from ubelix import run_gsea_batch
    
gsea_script_path = "../scripts/send_gsea_batch.sh"

n_cohorts = 20
all_N = [3,7,15]#,5,5,7,9,15]
gsea_methods = ["gseapy"] #["clusterORA_lfc","clusterORA","gseapy","gseapy_s2n"]
outlier_methods = ["none"] #,"jk"] #########,"pcah"]
DEA_methods = ["deseq2"] #["edgerqlf","edgerlrt"] ## reminder: do only edgerqlf for gsea S2N
libraries = ['GO_Biological_Process_2021', 'KEGG_2021_Human']

gsea_config_params = {
    
    "gsea_param_set": "p1", # id for this set of parameters
    "dea_param_set": "p1",
    "dea_param_set_lfc": "p2",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    "gsea_methods": gsea_methods,
    "libraries": libraries,
    "rankings": ["logFC"],
    
    "gsea_kwargs":  {
                    "gseapy": {"permutation_num": 200, "save_full_results": False, "threads": 4, "min_size": 15, "max_size": 500}
    }
}

gsea_config_params2 = {
    
    "gsea_param_set": "p2", # id for this set of parameters
    "dea_param_set": "p1",
    "dea_param_set_lfc": "p2",
    
    "overwrite": False, # overwrite existing tabs
    "data": str(outpath) + "/" + outname + ".csv",
    "outpath": str(outpath),
    "outname": outname,
    
    "DEA_methods": DEA_methods,
    "outlier_methods": outlier_methods,
    "gsea_methods": gsea_methods,
    "libraries": libraries,
    "rankings": ["logFC"],
    
    "gsea_kwargs":  {
                    "gseapy": {"permutation_num": 200, "save_full_results": False, "threads": 4, "min_size": 15, "max_size": 500}
    }
}

gsea_config_params2c = gsea_config_params2.copy()
gsea_config_params2c["dea_param_set_lfc"] = "2pc"

mode = "send jobs"
mode = "test main"
mode = "just testing"
sleep_seconds = 0

gsea_config_params = gsea_config_params2

run_gsea_batch(gsea_config_params, all_N, n_cohorts, libraries, gsea_script_path=gsea_script_path, mode=mode, sleep_seconds=sleep_seconds)

In [ ]:
!squeue -u pd21v747

In [ ]:
# data = "THCA"
# N = 3
# library = libraries[0]
# cohort = 1
# dea = "edgerqlf"
# out = "none"
# gsea_method = "clusterORA_lfc.fdr0.05.post_lfc1.lfc1"
# param_set = "p1"
# site = datasets[data]["site"]
# p=f"../data/{site}/{data}/{data}_N{N}/{data}_N{N}_{cohort:04}/gsea/{gsea_method}.{library}.{dea}.{out}.{param_set}.feather"
# t=open_table(p)
# print(len(t[t["FDR"]<0.05]))
# t.sort_values(by="FDR")

## Process jobs

In [ ]:
import sys, importlib
importlib.reload(sys.modules["process"])

from process import gsea_process_pipeline
gFDRs = [0.05]

#gsea_methods = ["clusterORA_lfc.fdr0.05.post_lfc1.lfc1", "gseapy","clusterORA.fdr0.05.post_lfc0.lfc0", "gseapy_s2n", "clusterORA.fdr0.05.post_lfc1.lfc0"]
#DEA_methods = ["edgerqlf", "deseq2", "edgerlrt"]
#gsea_process_pipeline(outpath, outname, all_N, DEA_methods, outlier_methods, gsea_methods, libraries, gFDRs, "p1", overwrite=1, overwrite_merged=0, n_cohorts=50, calculate_common=1)

gsea_methods = ["gseapy"]
DEA_methods = ["deseq2"]
all_N = [3,7,15] 
rankings = ["logFC"]

import subprocess as sp
output = sp.getoutput('squeue -u pd21v747')
jobs_running = output.find("send_gse") > 0

if not jobs_running:
    gsea_process_pipeline(outpath, outname, all_N, DEA_methods, outlier_methods, gsea_methods, libraries, rankings,
                          gFDRs, "p2", overwrite=1, overwrite_merged=0, n_cohorts=20, calculate_common=False)
    # when adding new method: overwrite merged and calculcate common only with new method, then only overwrite with all emthods

In [ ]:
# f="/storage/homefs/pd21v747/RNASeqReplicability/data/liver/LIHC/gsea_results.txt"
# with open(f, "rb") as ff:
#     gs = pickle.load(ff)
    
# gs

## Merge processed data sets

In [ ]:
done = ["lung"]
gsea_datasets = {sites[s]: {} for s in sites if s in done}

def load_gsea_results(path, site, name):
    resultsfile = f"{path}/{site}/{name}/gsea_results.txt"
    with open(resultsfile, "rb") as f:
        return pickle.load(f)
    
for s in sites:
    if s not in done: continue
    gsea_datasets[sites[s]]["site"] = s
    res = load_gsea_results(datapath, s, sites[s])
    for key in res.keys():
        gsea_datasets[sites[s]][key] = res[key]

method = "deseq2"
gsea_datasetsfile = f"../data/multi/gsea_datasets.{method}.txt"
pickler(gsea_datasets, gsea_datasetsfile)

In [ ]:
# Create tidy dataframe

DEAs = DEA_methods
FDRs = [0.05]
#all_N=[3,5,7,9,15]

    
quantity = ["median_rep", "median_terms","median_mcc0", "median_prec0",
            "median_mcc", "median_prec", "median_rec"]
#quantity += [q+"_common" for q in quantity]

iterables = [gsea_datasets,all_N,outlier_methods,DEAs,gsea_methods,libraries,rankings,FDRs,quantity]
multi_cols = pd.MultiIndex.from_product(iterables, names=["Data", "N", "Out", "DEA", "Enrichment", "Library", "Ranking", "FDR", "Val"])
combined = pd.DataFrame(columns=multi_cols)

ids = [] # id connecting samples from same dataset and cohort size, used for paired-sample testing pre vs. post outlier removal
for i, d in enumerate(gsea_datasets):
    for j, N in enumerate(all_N):
        k = i*len(all_N)+j 
        for out in outlier_methods:
            for dea in DEAs:
                for gsea in gsea_methods:
                    #if dea != "edger" and gsea_methods == "gsea_s2n": continue
                    for fdr in FDRs:
                        for lib in libraries:
                            for rnk in rankings:
                                for quant in quantity:
                                    try:
                                        col = (d,N,out,dea,gsea,lib,rnk,fdr,quant)
                                        combined.loc[0,col] = gsea_datasets[d][N][out][dea][gsea][lib][rnk][quant][fdr]
                                        if quant == "median_rep": ids.append(k)
                                    except KeyError:
                                        print(d,N,out,dea,fdr,rnk)
                                        assert 0
                                
combined_gsea_td = combined.unstack().unstack(level="Val").reset_index(level=["Data","N","Out","DEA","Enrichment","Library","Ranking","FDR"], drop=False)
combined_gsea_td["id"] = ids
combined_gsea_td.reset_index(drop=True, inplace=True)

# For outlier_method == "none", no adjustment is necessary, hence copy unadjusted values
#none_ix = combined_gsea_td[combined_gsea_td["Out"] == "none"].index # avoid setting with copy warning
#combined_gsea_td.loc[none_ix,"median_terms_adj"] = combined_gsea_td.loc[none_ix,"median_terms"]
#combined_gsea_td.loc[none_ix,"median_rep_adj"] = combined_gsea_td.loc[none_ix,"median_rep"]

for clean in cleanout:
    combined_gsea_td.loc[(combined_gsea_td[combined_gsea_td["Out"] == clean]).index, "Out"] = cleanout[clean]
for clean in cleandea:
    combined_gsea_td.loc[(combined_gsea_td[combined_gsea_td["DEA"] == clean]).index, "DEA"] = cleandea[clean]
combined_gsea_td.to_csv(f"../data/multi/combined_gsea_td.rev3.csv")
combined_gsea_td.head()

## Inspect results

In [ ]:
from misc import get_grid_size

y = "median_prec"
x = "N"

a  = combined_gsea_td
#a = a[a["Library"]=="KEGG_2021_Human"]
hues = ["Data","Library","Ranking","N"]
grid = get_grid_size(len(hues), k=0, fill=True)
fig, ax = plt.subplots(grid[0],grid[1],figsize=(6*grid[0],3*grid[1]), sharex=True,sharey=True)
ax=ax.flatten()

for i, hue in enumerate(hues):
    sns.scatterplot(data=a, x=x, y=y, hue=hue, ax=ax[i])    
    ax[i].invert_yaxis()
fig.tight_layout()

# Misc.

## Median replicates

In [ ]:
print("Data sets:", len(sites))

reps = dict()
for site in sites:
    f = datasets[sites[site]]["datapath"]
    reps[site] = (len(pd.read_csv(f, index_col=0).columns)) // 2
    print(site, reps[site])

In [ ]:
repsval = list(reps.values())
print(repsval)
print(np.median(repsval), f"[{np.min(repsval)}, {np.max(repsval)}]")

## Filter low-expressed genes

In [ ]:
from R_wrappers import filterByExpr_wrapper

f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEBP/BPLT/BPLT.csv"

outfile_filtered = f.replace(".csv","_filtered.csv")
filterByExpr_wrapper(inpath=f, outpath=outfile_filtered, design="unpaired")

## Test DEA

In [ ]:
from misc import paired_replicate_sampler

N = 15
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEBP/BPLT/BPLT.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/kidney/KIRC/KIRC.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/prostate/PRAD/PRAD.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/thyroid/THCA/THCA.csv"
#f = "../data/GSEPN/GIPF/GIPF.csv"

tab = pd.read_csv(f, index_col=0)

try:
    df = paired_replicate_sampler(tab, N)[0]
except:
    c = tab.columns.str.startswith("control").sum()
    m = min(c, tab.columns.str.startswith("ipf").sum())
    tab = tab.iloc[:,list(range(m))+list(range(c,c+m))]
    df = paired_replicate_sampler(tab, N)[0]
    
from DEA import normalize_counts
df_norm = normalize_counts(df)

In [ ]:
# design_file = "../data/thyroid/thyroid_meta.csv"

# design = pd.read_csv(design_file, index_col=0)
# # don't include days_to_birth since it is perfectly confounded with submitter_id
# design = design.T.rename({"case":"Condition"},axis=1)[["gender","submitter_id","Condition"]]
# #design["gender"] = ["M","F"] * (len(design)//2)


# design_file_N = "../data/test/test.design.csv"
# design = design.loc[df.columns]

# design

In [ ]:
# df_encoded = pd.get_dummies(design, columns=['submitter_id'], drop_first=True).drop(["gender","Condition"],axis=1)
# male_tumor = (design["gender"] == "male") & (design["Condition"] == "T")
# female_tumor = (design["gender"] == "female") & (design["Condition"] == "T")
# male_tumor.name = "Male.Tumor"
# female_tumor.name = "Female.Tumor"

# df_encoded = pd.concat([df_encoded, male_tumor, female_tumor], axis=1)
# df_encoded.index.name = None
# df_encoded

In [ ]:
# design_file = "../data/thyroid/thyroid_meta.csv"

# design = pd.read_csv(design_file, index_col=0)
# # don't include days_to_birth since it is perfectly confounded with submitter_id
# design = design.T.rename({"case":"Condition"},axis=1)[["gender","submitter_id","Condition"]]
# #design["gender"] = ["M","F"] * (len(design)//2)


# design_file_N = "../data/test/test.design.csv"
# design = design.loc[df.columns]


# df_encoded = pd.get_dummies(design, columns=['gender', 'submitter_id', 'Condition'], drop_first=True)

# # Add a constant column for the intercept
# df_encoded['intercept'] = True

# # Check the rank
# matrix = df_encoded.values
# rank = np.linalg.matrix_rank(matrix)
# full_rank = rank == matrix.shape[1]

# print(f"Rank: {rank}")
# print(f"Number of columns: {matrix.shape[1]}")
# print(f"Is full rank: {full_rank}")
# df_encoded

In [ ]:
# import pandas as pd
# import numpy as np

# s = np.random.choice(["M","F"],N)
# s = np.concatenate([s,s])
# # Example data
# data = {
#     'patient': [i for i in range(1, N+1)] + [i for i in range(1, N+1)],
#     'condition': ['N'] * N + ['T'] * N,
#     'sex': s
# }

# df = pd.DataFrame(data)

# # Ensure each patient has samples in both conditions
# assert df.groupby('patient')['condition'].nunique().eq(2).all()

# # Create the design matrix
# design = pd.get_dummies(df, columns=['patient', 'sex', 'condition'], drop_first=True)

# # Check rank
# rank = np.linalg.matrix_rank(design.values)
# ncols = design.shape[1]

# print(f"Rank: {rank}, Number of columns: {ncols}")
# design

In [ ]:
from DEA import run_dea

design_file = "../data/GSEPN/GIPF/GIPF.meta.csv"
design_file = "../data/thyroid/thyroid_meta.csv"

design = pd.read_csv(design_file, index_col=0)
design = design.T.rename({"case":"Condition","gender":"Sex","submitter_id":"Patient"},axis=1)[["Sex","Patient","Condition"]]


design_file_N = "../data/test/test.design.repeated.csv"
design = design.loc[df.columns]
design.to_csv(design_file_N)

outfile = f"/storage/homefs/pd21v747/RNASeqReplicability/data/test/test.controlled.N{N}.csv"
edgerqlf_kwargs = {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 0, "design": design_file_N,
                   "check_gof": False, "verbose":True}

run_dea(df, outfile, "edgerqlf", True, **edgerqlf_kwargs)

In [ ]:
outfile_uncontrolled = f"/storage/homefs/pd21v747/RNASeqReplicability/data/test/test.uncontrolled.N{N}.csv"
edgerqlf_kwargs = {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 0, "design": "paired", "check_gof": False,
                  "N_control": N, "N_treat": N, "verbose": True}
run_dea(df, outfile_uncontrolled, "edgerqlf", True, **edgerqlf_kwargs)

In [ ]:
all_N = [4,15]#[4,5,15,100]

fig, ax = plt.subplots(1,len(all_N),figsize=(4*len(all_N),4))

for i, n in enumerate(all_N):

    outfile = f"/storage/homefs/pd21v747/RNASeqReplicability/data/test/test.controlled.N{n}.csv"
    outfile_uncontrolled = f"/storage/homefs/pd21v747/RNASeqReplicability/data/test/test.uncontrolled.N{n}.csv"

    res = pd.read_csv(outfile, index_col=0)
    res2 = pd.read_csv(outfile_uncontrolled, index_col=0)

    ax[i].scatter(res["FDR"],res2["FDR"])
    ax[i].plot( (min(res["FDR"]),1), (min(res["FDR"]),1), ls="--",c="red")
    if n > 5:
        ax[i].set_xscale("log")
        ax[i].set_yscale("log")
    ax[i].set_xlabel("FDR uncontrolled")
    ax[i].set_ylabel("FDR controlled")
    ax[i].set_title(f"N={n}")
    
    #ax[i].set_xlim(min(min(res["FDR"]),min(res2["FDR"])), 1)
    #ax[i].set_ylim(min(min(res["FDR"]),min(res2["FDR"])), 1)
    
fig.tight_layout()

In [ ]:
from DEA import run_dea

outfile = "/storage/homefs/pd21v747/RNASeqReplicability/data/test/goftest.csv"

edgerqlf_kwargs = {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 0, "design": "paired", "check_gof": True}
        
run_dea(df, outfile, "edgerqlf", True, **edgerqlf_kwargs)

In [ ]:
res = pd.read_csv(outfile, index_col=0)
sig = res[res["FDR"]<0.05]
print(len(sig))
res

In [ ]:
sortedfit = gof.sort_values(by="x").index

In [ ]:
g = sortedfit[1]
print(res.loc[g])

plt.hist(df.loc[g])

In [ ]:
g = outfile.split("/")
g[-1] = "gof." + g[-1]
gof_file = "/".join(g)
gof = pd.read_csv(gof_file, index_col=0)

assert len(res) == len(gof)
gof.index = res.index

#gof.loc[sig.index].hist()
gof.hist()
plt.xlabel("pvalue goodness-of-fit")

In [ ]:
plt.scatter(res["FDR"], gof["x"], alpha=0.1)
plt.xlabel("FDR DEG")
plt.ylabel("pval goodness-of-fit")

## Wilcoxon

In [ ]:
from misc import paired_replicate_sampler

N = 3
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/GSEBP/BPLT/BPLT.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/kidney/KIRC/KIRC.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/prostate/PRAD/PRAD.csv"
f = "/storage/homefs/pd21v747/RNASeqReplicability/data/thyroid/THCA/THCA.csv"
#f = "../data/GSEPN/GIPF/GIPF.csv"

tab = pd.read_csv(f, index_col=0)

try:
    df = paired_replicate_sampler(tab, N)[0]
except:
    c = tab.columns.str.startswith("control").sum()
    m = min(c, tab.columns.str.startswith("ipf").sum())
    tab = tab.iloc[:,list(range(m))+list(range(c,c+m))]
    df = paired_replicate_sampler(tab, N)[0]

In [ ]:
from scipy.stats import ranksums, wilcoxon
from statsmodels.stats.multitest import multipletests
from DEA import normalize_counts

def wilcox_test(df_unnormalized, design, outfile="", overwrite=False):
    """Perform Wilcoxon rank sum test for each gene"""
    
    if design == "paired":
        if len(df_unnormalized.columns) % 2 != 0:
            raise Exception("df must have even number of columns for paired design")
        test_func = wilcoxon

    elif design == "unpaired":
        test_func = ranksums
    else:
        raise Exception("General design matrix not supported, must be paired or unpaired")
        
    if os.path.isfile(outfile) and not overwrite:
        logging.info("Existing file not overwritten")
        return
    
    df = normalize_counts(df_unnormalized)
    wilcoxon_results = {}
    for i, gene in enumerate(df.index):
        control_values = df.iloc[i, :len(df.columns)//2]
        treatment_values = df.iloc[i, len(df.columns)//2:]

        try:
            statistic, p_value = test_func(control_values, treatment_values)
        except ValueError:
            statistic, p_value = np.nan, np.nan
        
        wilcoxon_results[gene] = {'statistic': statistic, 'p_value': p_value}

    wilcoxon_results_df = pd.DataFrame.from_dict(wilcoxon_results, orient='index')
    p_values = wilcoxon_results_df['p_value']
    
    _, corrected_p_values, _, _ = multipletests(p_values.dropna(), method='fdr_bh')
    wilcoxon_results_df.loc[~wilcoxon_results_df['p_value'].isna(), 'FDR'] = corrected_p_values    

    if outfile != "":
        save_df(wilcoxon_results_df, outfile)
    return wilcoxon_results_df


def save_df(df, path):
    file_extension = path.split('.')[-1].lower()
    if file_extension == 'csv':
        df.to_csv(path)
    elif file_extension == 'feather':
        df.to_feather(path)
    else:
        raise ValueError("Unsupported file extension. Supported extensions are: .csv and .feather")

In [ ]:
from DEA import run_dea

outfile = "/storage/homefs/pd21v747/RNASeqReplicability/data/test/test.csv"
edgerqlf_kwargs = {"filter_expr": False, "cols_to_keep": ["logFC","logCPM","FDR"], "lfc": 0, "design": "paired"}
run_dea(df, outfile, "edgerqlf", True, **edgerqlf_kwargs)
res = pd.read_csv(outfile, index_col=0)
sig = res[res["FDR"]<0.05]
print(len(sig))

In [ ]:
outfile="/storage/homefs/pd21v747/RNASeqReplicability/data/test/wilcox_test_unpaired.csv"
wilcoxon_results_df_unpaired = wilcox_test(df, outfile=outfile, design="unpaired", overwrite=True)

outfile="/storage/homefs/pd21v747/RNASeqReplicability/data/test/wilcox_test.csv"
wilcoxon_results_df_pair = wilcox_test(df, outfile=outfile, design="paired", overwrite=True)

In [ ]:
sns.jointplot(x=wilcoxon_results_df_pair["FDR"], y=res["FDR"], kind="reg", line_kws=dict(color="r"))

In [ ]:
sns.jointplot(x=wilcoxon_results_df_unpaired["FDR"], y=res["FDR"], kind="reg", line_kws=dict(color="r"))

In [ ]:
sns.jointplot(x=wilcoxon_results_df_unpaired["FDR"], y=wilcoxon_results_df_pair["FDR"], kind="reg", line_kws=dict(color="r"))

In [ ]:
sig_wrs = wilcoxon_results_df[wilcoxon_results_df["FDR"]<0.05]
print(len(sig.index), len(sig_wrs), len(sig.index.intersection(sig_wrs.index)))

sig_wrs_norm = wilcoxon_results_df_norm[wilcoxon_results_df_norm["FDR"]<0.05]
print(len(sig.index), len(sig_wrs_norm), len(sig.index.intersection(sig_wrs_norm.index)))

len(sig_wrs.index.intersection(sig_wrs_norm.index))

### Formal

In [ ]:
df_norm = normalize_counts(df)

In [ ]:
control_values = df_norm.iloc[:, :len(df_norm.columns)//2] + 1
treatment_values = df_norm.iloc[:, len(df_norm.columns)//2:] + 1
control_values.columns = [c[1:] for c in control_values.columns]
treatment_values.columns = [c[1:] for c in treatment_values.columns]

control_values = np.log2(control_values)
treatment_values = np.log2(treatment_values)

lfc_thresh = 0
#treatment_values -= 2**lfc_thresh

_, p_values = wilcoxon(control_values, treatment_values, axis=1, alternative="greater")

_, corrected_p_values, _, _ = multipletests(p_values, method='fdr_bh')

In [ ]:
(control_values-treatment_values).mean()

In [ ]:
plt.hist(corrected_p_values)
(corrected_p_values < 0.05).sum()

In [ ]:
wilcoxon_results = {}
for i, gene in enumerate(df_norm.index):
    if i > 100: break
    control_values = df_norm.iloc[i, :len(df_norm.columns)//2]
    treatment_values = df_norm.iloc[i, len(df_norm.columns)//2:]

    try:
        statistic, p_value = wilcoxon(control_values, treatment_values)
    except ValueError:
        statistic, p_value = np.nan, np.nan

    wilcoxon_results[gene] = {'statistic': statistic, 'p_value': p_value}

wilcoxon_results_df = pd.DataFrame.from_dict(wilcoxon_results, orient='index')
p_values = wilcoxon_results_df['p_value']

_, corrected_p_values, _, _ = multipletests(p_values.dropna(), method='fdr_bh')
wilcoxon_results_df.loc[~wilcoxon_results_df['p_value'].isna(), 'FDR'] = corrected_p_values  

# Misc.

In [ ]:
from process import delete_redundant_slurmfiles

delete_redundant_slurmfiles(outpath, outname, all_N)            

## ZIP

tar -czf unpaired_folders.tar.gz $(find . -type d -name '*_unpaired')

In [ ]:
import os
import shutil
import zipfile

def zip_gsea_subdir(directory, target_subdir):
    # Define the full path to the gsea subdirectory
    gsea_dir = os.path.join(directory, target_subdir)

    # Check if the gsea subdirectory exists
    if os.path.exists(gsea_dir) and os.path.isdir(gsea_dir):
        # Define the path for the zip file
        zip_filename = os.path.join(directory, f"{target_subdir}.zip")
        
        try:
            # Zip the gsea folder
            with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
                # Walk through the gsea directory and add files to the zip
                for root, dirs, files in os.walk(gsea_dir):
                    for file in files:
                        full_path = os.path.join(root, file)
                        relative_path = os.path.relpath(full_path, start=gsea_dir)
                        zipf.write(full_path, arcname=relative_path)

            print(f"{target_subdir} subdir zipped successfully into {zip_filename}")

            # Remove the original gsea folder
            shutil.rmtree(gsea_dir)
            print(f"{target_subdir} subdir removed after zipping.")

        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print(f"{target_subdir} subdir does not exist in {directory}.")

In [ ]:
all_N = [3,4,5,6,7,8,9,10,12,15]
cancer_sites = {"liver": "LIHC",
         "thyroid": "THCA",
         "lung": "LUAD",
         "lung2": "LUSC",
         "kidney": "KIRC",
         "colorectal": "COAD",
         "breast": "BRCA",
         "prostate": "PRAD"}

cancer_sites_unpaired = {k+"_unpaired":v+"_unpaired" for k,v in cancer_sites.items()}

for d in cancer_sites_unpaired:
    outpath = f"../data/{d}/{cancer_sites_unpaired[d]}"
    for N in all_N:
        sub = f"{cancer_sites_unpaired[d]}_N{N}"
        outpath_n = os.path.join(outpath,sub)
        print(outpath_n)
        zip_gsea_subdir(outpath, sub)

In [ ]:
# all_N = [3,4,5,6,7,8,9,10,12,15]
# cohorts = range(1,101)

# for d in datasets:
#     outpath = datasets[d]["outpath"]
#     for N in all_N:
#         outpath_n = os.path.join(outpath,f"{d}_N{N}"
#         print(outpath_n)
#         #for cohort in cohorts:
#             #outpath_c = os.path.join(outpath,f"{d}_N{N}/{d}_N{N}_{cohort:04}")
#             #print(outpath_c)
#             #zip_gsea_subdir(outpath_c, "gsea")

In [ ]:
import sys, importlib
importlib.reload(sys.modules["enrichment"])


In [ ]:
for file in BRCA.*; do
    new_name="BRCA_unpaired.${file#*.}"
    mv "$file" "$new_name"
done